In [ ]:
import gurobipy as gp
import numpy as np
from gurobipy import GRB

In [ ]:
zu = np.inf
zl = 0

In [ ]:
A = np.array([
    [3, 1, 2, 4, 1],
    [2, 3, 1, 4, 2]
])
b = np.array([6, 5])

In [ ]:
np.sum(A, axis=-1)

In [ ]:
A.T[0]

In [ ]:
lambd = np.zeros(A.shape[0])
lambd

In [ ]:
Ct = [sum([(1 - lambd[r]) * A[r][c] for r in range(b.shape[0])]) 
      for c in range(A.shape[-1])]
Ct

In [ ]:
x = [0 if ct >= 0 else 1 for ct in Ct]
x

In [ ]:
C_t = (1 - lambd) @ A
x = np.where(C_t < 0, 1, 0)
C_t

In [ ]:
L = C_t @ x + lambd @ b
L

In [ ]:
zl = max(zl, L)
zl

In [ ]:
g = b - A @ x
g

In [ ]:
A = np.array([
    [1, 2, 4],
    [3, 1, 4],
    [2, 2, 2]
])
b = np.array([2, 5, 1])

In [ ]:
def subgrad_opt(
        A, b, z_ub, z_lb, lambd, f=2, k=15, eps=0.005, omega=500):
    # TODO: z_ub cannot be inf here
    unchanged = 0
    t = 0
    lambd_best = lambd
    x_best = None
    while (z_ub > z_lb):
        c = (1 - lambd) @ A
        # print("c: ", c)
        x = np.where(c < 0, 1, 0)
        # print(x)
        L = c @ x + lambd @ b
        # print("L: ", L)
        g = b - A @ x
        if L > z_lb:
            z_lb = L
            lambd_best = lambd
            x_best = x
            unchanged = 0
            # print("L new: ", L)
        else:
            unchanged += 1
        if unchanged == k:
            unchanged = 0
            f /= 2
        sigma = f * (z_ub - z_lb) / np.linalg.norm(g) ** 2
        lambd = np.maximum(np.zeros_like(lambd), lambd + sigma * g)
        # lambd = np.minimum(1, lambd)
        # print("lambd: ", lambd)
        t += 1
        if f < eps or t > omega:
            print(f, t, g)
            break
    
    return lambd_best, z_lb, x_best, f

In [ ]:
subgrad_opt(A, b, 100, 0, np.zeros(A.shape[0]))

# RCH

In [ ]:
def rch(A, b):
    lambd = np.zeros(A.shape[0])
    x = np.zeros(A.shape[-1])
    s = (1 - lambd) @ A
    print("s", s)
    for j in range(A.shape[-1]):
        if s[j] == 0:
            x[j] = 1
        else:
            x[j] = 0

    print("x", x)
    
    for i in range(A.shape[0]):
        N_i = np.where(A[i] > 0)[0]
        print("i N_i", i, N_i)
        if sum([A[i][j] for j in N_i]) < b[i]:
            s_k = min(s[N_i])
            lambd[i] += s_k
            for j in N_i:
                s[j] -= s_k
                if s[j] == 0:
                    x[j] = 1

    print(s, x)

    return lambd, make_prime(x, A, b)

# RCH not working because it assumes that only one column needs to cover

def make_prime(x, A, b):
    x1 = np.where(x == 1)[0]
    for j in x1:
        new_x1 = x1[~np.isin(x1, [j])]
        if A @ new_x1 >= b:
            x1 = new_x1
            print("changed")
    return x1


In [ ]:
rch(A, b)

# Solve dual

In [2]:
import gurobipy as gp
import numpy as np
from gurobipy import GRB

In [57]:
A = np.array([
    [3, 1, 2, 4, 1],
    [2, 3, 1, 4, 2],
    [1, 1, 2, 2, 5],
    [2, 2, 4, 5, 2]
])
b = np.array([6, 5, 4, 8])

In [58]:
x1 = [0]
x0 = [1]
db = np.copy(b)
for j in x1:
    db -= A[:,j]
db

array([3, 3, 3, 6])

In [59]:
c = np.sum(A, axis=0)

In [ ]:
for j in x0:
    c[j] = 0
c

In [60]:
md = gp.Model("dual")
y = md.addMVar(A.shape[0], lb=0, name="y")
md.setObjective(db @ y, GRB.MAXIMIZE)
constr = md.addConstr(A.T @ y <= c)

In [61]:
for j in x1 + x0:
    md.remove(constr[j])

In [62]:
md.optimize()
lambd = []
for v in md.getVars():
    lambd.append(v.x)
lambd = np.array(lambd)
md.getObjective().getValue(), lambd

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9700K CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 4 columns and 12 nonzeros
Model fingerprint: 0x8b45a627
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [3e+00, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+00, 2e+01]
Presolve time: 0.01s
Presolved: 3 rows, 4 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1000000e+31   7.500000e+30   2.100000e+01      0s
       3    1.5765957e+01   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.576595745e+01


(15.76595744680851, array([0.        , 1.5106383 , 0.80851064, 1.46808511]))

In [64]:
rc = (1 - lambd) @ A
x = np.where(rc < 0, 1, 0)
x[x1] = 1
x[x0] = 0

(db @ lambd, rc @ x + lambd @ b)

(15.76595744680851, 23.76595744680851)

In [65]:
s = [c[j] for j in x1]
(db @ lambd - (rc @ x + lambd @ b), s)

(-8.0, [8])

In [ ]:
f=2 
k=5 
eps=0.005 
omega=500
ub = 90
x0 = []
x1 = []
lambd = np.zeros(A.shape[0])
lb = -np.inf

unchanged = 0
t = 0
lambd_best = lambd
x_best = None
while (ub > lb):
    c = (1 - lambd) @ A
    x = np.where(c < 0, 1, 0)
    x[x0] = 0
    x[x1] = 1
    L = c @ x + lambd @ b
    g = b - A @ x
    if L > lb:
        lb = L
        lambd_best = lambd
        x_best = x
        unchanged = 0
    else:
        unchanged += 1
    if unchanged == k:
        unchanged = 0
        f /= 2
    sigma = f * (ub - lb) / np.linalg.norm(g) ** 2
    lambd = np.maximum(
        np.zeros_like(lambd), lambd + sigma * g)
    t += 1
    if f < eps or t > omega:
        break

In [ ]:
lambd_best

# Dual ascent

In [ ]:
c = np.sum(A, axis=0)
# s = (1 - lambd) @ A
s = c
lambd = np.zeros((A.shape[0]))
N = []
for i in range(A.shape[0]):
    idx = np.where(A[i] > 0)[0]
    N.append((i, idx))
N


In [ ]:
list.sort(N, key=lambda x: x[0])
print(N)

In [ ]:
for i, _ in N:
    lambd[i] = min([s[j] for j in N[i][1]])
    for j in range(A.shape[-1]):
        if j in N[i][1]:
            s[j] -= lambd[i]
lambd

# Dual ascent 2

In [ ]:
c = np.sum(A, axis=0)
lambd = np.zeros((A.shape[0]))
s = (1 - lambd) @ A
N = []
for i in range(A.shape[0]):
    idx = np.where(A[i] > 0)[0]
    N.append((i, idx))

list.sort(N, key=lambda x: x[0], reverse=True)

for i, _ in N:
    lambd[i] = max(0, lambd[i] + min(0, min([s[j] for j in N[i][1]])))

list.sort(N, key=lambda x: x[0])

for i, _ in N:
    lambd[i] = lambd[i] + min([s[j] for j in N[i][1]])

lambd